In [1]:
from pathlib import Path
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder
import pickle
import bz2
import matplotlib.pyplot as plt
import numpy as np
import docx2txt

In [2]:
data_dir = Path('data/prepared')
!ls {data_dir}

09_prepared.pkl.bz2  10_prepared.pkl.bz2  11_prepared.pkl.bz2


In [3]:
one = data_dir / '09_prepared.pkl.bz2'
two = data_dir / '10_prepared.pkl.bz2'
three = data_dir / '11_prepared.pkl.bz2'
products_path = 'data/product_groups.docx'
products_path_csv = 'data/product_groups.csv'

In [4]:
text = docx2txt.process(products_path)
with open(products_path_csv, 'w+') as f:
    f.write(text)

df_prod = pd.read_csv(products_path_csv, sep=';')
df_prod

,id_tov,id_group,name_group
0,52,10004,Кисломолочные продукты
1,75,10004,Кисломолочные продукты
2,77,10004,Кисломолочные продукты
3,143,10004,Кисломолочные продукты
4,151,10004,Кисломолочные продукты
...,...,...,...
5645,40762,52194,Подарки и сувениры
5646,40763,52194,Подарки и сувениры
5647,40764,52194,Подарки и сувениры
5648,40765,52194,Подарки и сувениры


In [5]:
with bz2.open(one, 'rb') as f:
    df1 = pickle.load(f)
df1.head()

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_card_int,id_doc_int
0,2020-09-01 17:01:25,0E05D308-5CEC-EA11-B444-005056A7539A,0,8594499,52,271,1.0,108.00,False,1538855,1283228
1,2020-09-01 17:35:36,610205A1-61EC-EA11-B444-005056A7539A,0,1452388,52,271,1.0,108.00,False,267307,8873113
2,2020-09-01 19:19:21,8AF19602-70EC-EA11-B444-005056A7539A,0,3493538,52,271,1.0,107.95,False,610220,12712899
3,2020-09-01 21:53:34,EB6C71A3-84EC-EA11-B444-005056A7539A,0,2491281,52,271,1.0,108.00,False,441497,21535283
4,2020-09-01 18:42:31,0706023F-6BEC-EA11-B444-005056A7539A,0,5732396,61,97,2.0,88.00,False,1065358,642341


In [ ]:
with bz2.open(two, 'rb') as f:
    df2 = pickle.load(f)
df2.head()

In [ ]:
with bz2.open(three, 'rb') as f:
    df3 = pickle.load(f)
df3.head()

First, I looked if there is some noise on the lower end. Like is some values are strange, and ,who would've thought, there are. Sum has 69 orders that are less then 0 ( it also has 20k+ orders with sum == 0, but it seems to be a feature). Also quantity can be 0 or lower. These may be just noize, errors or some special mark of inter-store transactions (the ones with sum==0 may be same, but there are too much of them to ignore
).

In [40]:
df1[['id_card_int','quantity']].groupby('id_card_int').mean().describe()
# here we can strange min and max, that are clearly outliers

,quantity
count,1.358089e+06
mean,1.403014e+00
std,1.189936e+01
min,3.400000e-02
25%,1.000000e+00
50%,1.053500e+00
75%,1.250000e+00
max,2.500500e+03


In [39]:
df1[['id_card_int','sum']].groupby('id_card_int').mean().describe()

,sum
count,1.358089e+06
mean,1.191979e+02
std,6.456538e+01
min,0.000000e+00
25%,8.558579e+01
50%,1.095796e+02
75%,1.380000e+02
max,7.760747e+03


In [20]:
df1[df1['sum'] < 0]

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_card_int,id_doc_int
4206092,2020-09-04 09:10:21,827B9591-75EE-EA11-B444-005056A7539A,0,7009929,23074,19269,1.268,-470.43,False,1301664,11939194
4206102,2020-09-04 09:09:24,F6807973-75EE-EA11-B444-005056A7539A,0,7707981,23074,19269,1.268,-470.43,False,1419650,22549976
4210124,2020-09-04 08:37:00,49C21AF5-70EE-EA11-B444-005056A7539A,0,5554550,23074,19269,1.084,-402.16,False,1035577,6745380
4218019,2020-09-04 08:43:37,B681FEE3-71EE-EA11-B444-005056A7539A,0,0643258,23074,19269,0.001,-0.37,False,126995,16694740
4218021,2020-09-04 08:43:37,B681FEE3-71EE-EA11-B444-005056A7539A,0,0643258,23074,19269,2.375,-881.13,False,126995,16694740
...,...,...,...,...,...,...,...,...,...,...,...
5396780,2020-09-04 08:37:49,1CC214FB-70EE-EA11-B444-005056A7539A,0,4916790,23074,19269,1.236,-458.56,False,906517,2630312
5438901,2020-09-04 09:28:21,E6F17E03-78EE-EA11-B444-005056A7539A,0,5931979,23074,19269,0.572,-212.21,False,1102820,21125316
5450157,2020-09-04 09:21:17,CD1BCB2C-77EE-EA11-B444-005056A7539A,0,3290674,23074,19269,0.915,-339.47,False,574393,18763274
5450159,2020-09-04 09:21:17,CD1BCB2C-77EE-EA11-B444-005056A7539A,0,3290674,23074,19269,0.915,-339.47,False,574393,18763274


In [42]:
df1[df1['sum'] < 0]['id_card_int'].unique()

array([1301664, 1419650, 1035577,  126995,  417317,  134497,  499226,
        707815, 1374348,  611055, 1348391, 1434934, 1837262,  388857,
       1273966, 1789797, 1666466,  867354,  609910,  853388,  768501,
        748956, 1694538,  258353,  430505, 1308696,  902664,  568502,
        308154,  605762,  886656,   43295,  253299,  150211,  219744,
        986409, 1192787,  526910, 1247073,  733858, 1827079,  472833,
         48370, 1425778,  144164,   85962,   28238, 1485723,  718534,
        116276, 1588589,  446374,  100408, 1538576,  906517, 1102820,
        574393,  322028])

In [13]:
df1[df1['quantity']<=0]

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_card_int,id_doc_int
76223,2020-09-01 19:48:39,FB34A539-74EC-EA11-B444-005056A7539A,5588920,0586045,15738,19687,0.0,0.79,False,116224,22979294
144688,2020-09-01 17:10:48,4089BF02-5DEC-EA11-B444-005056A7539A,0,A485529,28662,19687,0.0,0.89,False,1741530,5901464
1211336,2020-09-01 19:50:39,2FD6C495-74EC-EA11-B444-005056A7539A,0,5259037,23008,17616,0.0,0.51,False,974892,4375162
1649328,2020-09-02 22:07:29,40D624F8-4FED-EA11-B444-005056A7539A,0,0761743,731,17268,0.0,0.14,False,145090,5928667
2013148,2020-09-02 16:28:46,4D29BB5A-20ED-EA11-B444-005056A7539A,0,0988768,731,21309,0.0,0.18,False,183930,7056809
...,...,...,...,...,...,...,...,...,...,...,...
40844727,2020-09-30 20:07:33,24CECE9A-3F03-EB11-B444-005056A7539A,0,0139028,22911,17394,0.0,0.23,False,29462,3367001
40845729,2020-09-30 20:07:33,24CECE9A-3F03-EB11-B444-005056A7539A,0,0139028,37885,19438,0.0,0.38,False,29462,3367001
41157457,2020-09-30 17:58:33,CDA17091-2E03-EB11-B444-005056A7539A,0,3807306,22911,17394,0.0,0.46,False,670914,18810832
41705116,2020-09-30 13:05:17,0B27EEE3-0403-EB11-B444-005056A7539A,0,4176864,15817,15255,0.0,0.22,False,748972,1020540


In [43]:
df1[df1['quantity'] <= 0]['id_card_int'].unique()

array([ 116224, 1741530,  974892,  145090,  183930, 1453506, 1268126,
        792937, 1126758, 1765037, 1301664, 1419650, 1035577,  126995,
        417317,  134497,  499226,  707815, 1374348,  611055, 1348391,
       1434934, 1837262,  388857, 1273966, 1789797, 1666466,  867354,
        609910,  853388,  768501,  748956, 1694538,  258353, 1673423,
        430505, 1308696,  902664,  568502,  308154,  605762,  886656,
         43295,  253299,  150211,  219744,  986409, 1192787,  526910,
       1247073,  733858, 1827079,  472833,  302382,   48370, 1425778,
        144164,   85962,   28238, 1485723,  718534,  116276, 1588589,
        446374,  100408, 1538576,  906517, 1102820,  574393,  894939,
        599030,  812051,  804951, 1682205, 1305495,  301680,  552414,
       1430504,  197671,  987507, 1368638,  244598, 1185383, 1061023,
         16609,  757237,  940826, 1901172, 1379458,  253108,  335958,
       1858483,  230994, 1226065,  840555,   12751,  202158, 1821422,
        392789, 1494

Next, I looked at the higher end.

Here we see the ones with large quantities and zero sum. Seems like an inter-store for me.

In [48]:
df1[(df1['quantity']>1000) & (df1['sum']==0)]

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_card_int,id_doc_int
27520,2020-09-01 13:32:18,EDEB3984-3FEC-EA11-B444-005056A7539A,0,4441279,1113,-9999,5000.0,0.0,False,806096,21763622
37567,2020-09-01 15:53:48,34DFE7D7-52EC-EA11-B444-005056A7539A,0,7828592,1113,-9999,2000.0,0.0,False,1438102,4834001
43897,2020-09-01 12:51:22,E913E777-39EC-EA11-B444-005056A7539A,0,0357110,1113,-9999,5000.0,0.0,False,72344,21320546
57565,2020-09-01 21:20:20,9D9C911C-80EC-EA11-B444-005056A7539A,0,2415958,1113,-9999,5000.0,0.0,False,427981,14419056
69522,2020-09-01 11:14:28,5ED8B16A-2BEC-EA11-B444-005056A7539A,0,8505515,1113,-9999,4000.0,0.0,False,1528414,8674916
...,...,...,...,...,...,...,...,...,...,...,...
42051074,2020-09-30 21:32:58,9ED8426C-4C03-EB11-B444-005056A7539A,0,2336308,1113,-9999,1500.0,0.0,False,414365,14532371
42082068,2020-09-30 15:02:54,2452AABF-1503-EB11-B444-005056A7539A,0,3937328,1113,-9999,3000.0,0.0,False,697590,3322572
42082069,2020-09-30 17:27:05,4B5E1B0D-2A03-EB11-B444-005056A7539A,0,3937328,1113,-9999,5000.0,0.0,False,697590,6892522
42092816,2020-09-30 12:27:49,EF500E71-FF02-EB11-B444-005056A7539A,0,0211831,1113,-9999,2000.0,0.0,False,44611,21891204


In [55]:
df1[(df1['quantity']>1000) & (df1['sum']==0)]['id_card_int'].unique()

array([ 806096, 1438102,   72344, ..., 1421253,  414365,  748344])

Here we can see one individual who has obscene sum and many transactions per day.

In [51]:
s = df1[['sum', 'id_card_int']].groupby('id_card_int').sum()

In [54]:
s.sort_values('sum')

,sum
id_card_int,
1871302,0.00
527413,0.00
1843017,0.00
405371,0.00
365257,0.00
...,...
1447941,98215.49
1755035,116411.20
1243870,124131.91


In [57]:
df1[df1['id_card_int']==883386]

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_card_int,id_doc_int
1221,2020-09-01 13:41:07,AEEFB494-40EC-EA11-B444-005056A7539A,0,4806534,20813,14782,1.0,168.0,False,883386,16003441
1853,2020-09-01 18:51:35,8A2AD918-6CEC-EA11-B444-005056A7539A,0,4806534,27495,14911,1.0,34.0,False,883386,12641780
1859,2020-09-01 18:04:23,FEDAE894-65EC-EA11-B444-005056A7539A,0,4806534,27695,15385,2.0,152.0,False,883386,23313124
1925,2020-09-01 18:51:35,8A2AD918-6CEC-EA11-B444-005056A7539A,0,4806534,29568,13354,1.0,36.0,False,883386,12641780
1976,2020-09-01 18:04:23,FEDAE894-65EC-EA11-B444-005056A7539A,0,4806534,30560,13614,1.0,17.0,False,883386,23313124
...,...,...,...,...,...,...,...,...,...,...,...
42100270,2020-09-30 17:11:56,AB13A45F-2703-EB11-B444-005056A7539A,0,4806534,28799,15916,1.0,130.0,False,883386,15650853
42100329,2020-09-30 18:29:17,A586484B-3303-EB11-B444-005056A7539A,0,4806534,30218,12554,1.0,185.0,False,883386,15143032
42100460,2020-09-30 17:11:56,AB13A45F-2703-EB11-B444-005056A7539A,0,4806534,35061,20817,1.0,150.0,False,883386,15650853
42100484,2020-09-30 15:55:39,D589FC26-1D03-EB11-B444-005056A7539A,0,4806534,36157,12638,1.0,190.0,False,883386,19534242


In [74]:
res=list(df1[(df1['quantity']>1000) & (df1['sum']==0)]['id_card_int'].unique())+[883386]+list(df1[df1['quantity'] <= 0]['id_card_int'].unique())+list(df1[df1['sum'] < 0]['id_card_int'].unique())
res[:5]

[806096, 1438102, 72344, 427981, 1528414]

In [79]:
res = list(set(res))

In [82]:
df_res = pd.DataFrame({'id_card_int':res})
df_res

,id_card_int
0,311297
1,528385
2,139270
3,1679367
4,1310735
...,...
1926,471031
1927,978937
1928,1368058
1929,352251


In [85]:
with bz2.open("result.pkl.bz2", 'wb') as f:
    pickle.dump(df_res, f)

In [86]:
with bz2.open("result.pkl.bz2", 'rb') as f:
    df_res_l = pickle.load(f)
df_res_l.head()

,id_card_int
0,311297
1,528385
2,139270
3,1679367
4,1310735
